In [1]:
# 네이버에서 검색어 입력받아 검색 한 후 블로그 메뉴를 선택하고
# 오른쪽에 있는 검색옵션 버튼을 눌러서
# 정렬 방식과 기간을 입력하기

#Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys # 시스템
import os  # 시스템

import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리

from bs4 import BeautifulSoup    # html 데이터를 전처리
from selenium import webdriver   # 웹 브라우저 자동화
import time    # 서버와 통신할 때 중간중간 시간 지연
from tqdm.notebook import tqdm   # for문 돌릴 때 진행상황을 %게이지로 알려준다.

# 책 예제 따라가기

In [ ]:
엑셀 url 합치기
취미운동 다시 뽑기>홈트 ~ url 합치기>duplicate

In [7]:
import pandas as pd
blog_df = pd.DataFrame([])
folder = './naver/'
f_list = ['취미운동_url.xlsx',
          'blog_운동(홈트~).xlsx']

for fname in f_list:
    fpath = folder + fname
    temp = pd.read_excel(fpath,engine='openpyxl')
    blog_df = blog_df.append(temp)


In [8]:
blog_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9203 entries, 0 to 8202
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     9203 non-null   object
 1   title   9203 non-null   object
dtypes: object(2)
memory usage: 215.7+ KB


In [9]:
blog_df.drop_duplicates(subset = ['title'], inplace=True)

In [10]:
blog_df.info() # 20개

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9183 entries, 0 to 8202
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     9183 non-null   object
 1   title   9183 non-null   object
dtypes: object(2)
memory usage: 215.2+ KB


In [11]:
blog_df.to_excel('./naver/0515_운동_url.xlsx', index=False)

In [13]:
url_load = pd.read_excel("./naver/0515_운동_url.xlsx",engine='openpyxl')

num_list = len(url_load)

print(num_list)
url_load

9183


,url,title
0,https://blog.naver.com/wonnoboo2?Redirect=Log&...,[부산 폴댄스] 대한폴댄스연맹에서 새로운 취미 운동!
1,https://blog.naver.com/blooming_365?Redirect=L...,"광주 필라테스, 벌써 10개월째 취미 운동"
2,https://blog.naver.com/khu_fullmoon?Redirect=L...,"다시 시작한 취미운동ㅣ초급 해마, 토네이도 후기..."
3,https://blog.naver.com/ywk6kies?Redirect=Log&l...,답십리발레 장안동 직장인취미운동 추천! (존잼)
4,https://blog.naver.com/khm2878?Redirect=Log&lo...,"나의 취미운동 - 대전폴댄스 , 둔산동폴댄스학원 , 맨몸운동"
...,...,...
9178,https://blog.naver.com/xox_dorothy?Redirect=Lo...,복대동 필라테스 오랜만에 다녀온 후기
9179,https://blog.naver.com/imspinning?Redirect=Log...,간석 필라테스 오늘도 건강해지기!
9180,https://blog.naver.com/ddxqcn58?Redirect=Log&l...,신림역 필라테스 요기서 첫 운동하세요!
9181,https://blog.naver.com/lavender130?Redirect=Lo...,필라테스가 그렇게 좋은 운동... 비교사진 & 필라테스의 비밀)


# 좋아요 + 댓글 테이블 구상하기

In [14]:
import time
import re

start = time.time()

dict = {}    # 전체 크롤링 데이터를 담을 그릇

number = 1  # 수집할 글 갯수 정하기

# 수집한 url 돌면서 데이터 수집
for i in tqdm(range(0, number)): # 끊긴 경우 여기 숫자만 변경하면 됨
    # 글 띄우기
    url = url_load['url'][i]
    driver = webdriver.Chrome('C:\chrome\chromedriver.exe')  # 윈도우는 "chromedriver.exe"
    driver.get(url)   # 글 띄우기
    
    # 크롤링
    
    try : 
        # iframe 접근
        driver.switch_to.frame('mainFrame')

        target_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성
        
        # 제목 크롤링 시작
        overlays = ".se-module.se-module-text.se-title-text"                                 
        tit = driver.find_element_by_css_selector(overlays)          # title
        title = tit.text
        title = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9| ]", " ", title)
        
        # '댓글' 클릭
        driver.find_element_by_link_text(" 댓글").click( )
        time.sleep(1)
        
        # 댓글 크롤링
        
        for comment in comments:
            try:
                nickname = driver.find_element_by_css_selector('.u_cbox_nick')[0].text
                datetime = driver.find_element_by_css_selector('.u_cbox_date')[0].text
                content = driver.find_elements_by_css_selector('.u_cbox_contents')[0].text
                content_str = ' '.join(comment)
                content_str = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9|.|,| ]", " ", content_str)
            except:
                pass
        time.sleep(2)
        
        # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
        dict[i] = comment_info
        time.sleep(1)
        
        # 크롤링이 성공하면 글 제목을 출력하게 되고,
        print(i, title)

        # 글 하나 크롤링 후 크롬 창을 닫습니다.
        driver.close()       
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동합니다.
    except:
        driver.close()
        time.sleep(1)
        continue
        
    # 중간,중간에 파일로 저장하기
    if i == 750 or 900 :
        # 판다스로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_excel("test.xlsx", encoding='utf-8-sig')
        time.sleep(3)

print('수집한 글 갯수: ', len(dict))
print(dict)

end = time.time() - start
print("used time: %dm%ds" % divmod(end,60))


  0%|          | 0/1 [00:00<?, ?it/s]

수집한 글 갯수:  0
{}
used time: 0m8s


In [ ]:
import time
import re

start = time.time()

dict = {}    # 전체 크롤링 데이터를 담을 그릇

number = 1000  # 수집할 글 갯수 정하기

# 수집한 url 돌면서 데이터 수집
for i in tqdm(range(625, number)): # 끊긴 경우 여기 숫자만 변경하면 됨
    # 글 띄우기
    url = url_load['url'][i]
    driver = webdriver.Chrome('C:\chrome\chromedriver.exe')  # 윈도우는 "chromedriver.exe"
    driver.get(url)   # 글 띄우기
    
    # 크롤링
    
    try : 
        # iframe 접근
        driver.switch_to.frame('mainFrame')

        target_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성
        
        # 제목 크롤링 시작
        overlays = ".se-module.se-module-text.se-title-text"                                 
        tit = driver.find_element_by_css_selector(overlays)          # title
        title = tit.text
        title = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9| ]", " ", title)
        
        # 글쓴이 크롤링 시작
        overlays = ".nick"                                 
        nick = driver.find_element_by_css_selector(overlays)         # nickname
        nickname = nick.text

        # 날짜 크롤링
        overlays = ".se_publishDate.pcol2"                                 
        date = driver.find_element_by_css_selector(overlays)         # datetime
        datetime = date.text
        datetime_str = ' '.join(datetime)

        # 좋아요 크롤링
        overlays = '.u_cnt.count'
        like = driver.find_element_by_css_selector(overlays)
        likecount = like.text
        
        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"                                 
        contents = driver.find_elements_by_css_selector(overlays)    # contents

        # 블로그 
        content_list = []
        for content in contents:
            content_list.append(content.text)
             
        content_str = ' '.join(content_list) # 하나 이상의 공백을 치환           # content_str
        content_str = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9|.|,| ]", " ", content_str)
        
        # 글 하나는 target_info라는 딕셔너리에 담기게 되고,
        target_info['title'] = title
        target_info['nickname'] = nickname
        target_info['datetime'] = datetime_str
        target_info['content'] = content_str
        target_info['url'] = url

        # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
        dict[i] = target_info
        time.sleep(1)
        
        # 크롤링이 성공하면 글 제목을 출력하게 되고,
        print(i, title)
        
        # 댓글 테이블
        comment_list = []
        for comment in comments:
            comment_list.append(comment.text)
            
        comment_str = ' '.join(comment_list)
        content_str = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9|.|,| ]", " ", comment_str)

        # 글 하나 크롤링 후 크롬 창을 닫습니다.
        driver.close()       
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동합니다.
    except:
        driver.close()
        time.sleep(1)
        continue
    
    # 중간,중간에 파일로 저장하기
    if i == 750 or 900 :
        # 판다스로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_excel("운동 크롤링 결과.xlsx", encoding='utf-8-sig')
        time.sleep(3)

print('수집한 글 갯수: ', len(dict))
print(dict)

end = time.time() - start
print("used time: %dm%ds" % divmod(end,60))

  0%|          | 0/375 [00:00<?, ?it/s]

625 5060 파고드는 언택트 서비스   쇼핑에서 운동   취미까지 뻗어간다
626  강서구 발산역 발레  승무원들이 많이 배우는 취미 운동 
627 광화문PT 마포 피티 운동은 새로운 삶의 시작 종로 을지로 
628 하루만에 1kg 감량  최강 댄스유산소운동   라틴피트니스 
629 인천어린이리틀야구클럽 인천남동구BC야구단 2월마지막째주 평일훈련   인천남동구유소년야구단 SSG프로야구단 인천연수구어린이야구교실 인천취미야구클럽 인천초등엘리트야구 시흥배곧학생야구 
630  운동 취미  헬스초보의 13일차 운동기록 
631 운동을 취미삼기  Working out as a hobby 
632 집 취미 생활추천 습관잡기소셜모임 CGS
633 중학생  복싱1년차  open gym 트레이닝 운동 취미 청소년 체력관리 학생 스트래스해소 신봉동복싱 성복동복싱 풍덕천동복싱 동천동복싱 상현동복싱 수지구청복싱
634 100  취미로 운동하는 성인부들의 태권도 품새 실력은     부천 성인태권도 
635 천안 어린이승마 성인승마 취미추천 주몽승마장
636 다산신도시운동 인생취미 비복싱짐에서 만들기
637  가족취미운동  요가듀엣레슨 천안신방통정지구 우리가족 새로운 취미
638 코로나19 시대  더 필요한 주거공간은  취미 휴식 운동
639  헤공일상  또 다른 취미가 생겼어요
640  GDR 인천구월점  성인  직장인 취미활동 취미생활 추천 운동 추천
641  취미  성인발레 
642 취미로 하는 운동 폴댄스  부산 해운대 달폴댄스 아카데미  dalpole academy  
643 운동이 취미가 됐으면 좋겠다
644  일상운동  직장인 퇴근 후 운동 Life 취미생활 추천 합니다 
645 20대 여자 운동 추천   취미 시작한 배드민턴 소모임 퍼포먼스  
646 어른이날 주짓수충청북도대표 동남주짓수 동남지구 동남지구주짓수 청주주짓수 용암동주짓수 금천동주짓 파라에스트라동남 취미 다이어트 여성호신술 운동 자영업자 자기방어 청주주짓수 주짓수
647  엘에스발레  신월동 발레 양천구 발

826  취미 운동 임페리얼레이크 야간 라운딩
827 취미 찾기 2일차
828  공덕 헬스장    공덕 헬스  운동에 취미 붙이기 
829 광주 동구 킥복싱 무에타이 운동 취미 다이어트 체육관 조대 정문 팀맥짐     여름맞이 다이어트 함께해요 동명동 서석동 지산동 산수동 학동 양림동 다이어트 운동 킥복싱 체육관  
830 내 나이 50에 하고 있는 운동과 취미   
831  06 02 운동과 카페에서 취미생활
832  발레 EJ이화발레학원 장안점 성인취미발레 답십리발레학원 스트레칭하기 좋은운동
833 초급폴 초등폴 학생 조아라댄스아카데미 폴댄스 사이드슈퍼맨 스핀폴 초급반 취미반 취미폴 재밌는운동 강서구폴댄스 화곡동폴학원 엔젤스핀
834 별별취미 강좌  운동처방 및 상담  후기
835 임영웅  운동 취미  슬기로운 자가격리  유튜브로 팬과 소통
836  강동구 유도관  초심자도 쉽게 배우는 유도    강동유도관   성인유도   강동구 헬스   체대입시   천호 헬스   강동구 취미 운동   강동구 주짓수   강동구 다이어트 운동
837 부모님여행 노는법에서 취미  여행프로그램 선물하기 
838 임영웅  운동 취미  슬기로운 자가격리  유튜브로 팬과 소통
839 운동이나 취미 공부 등 나를 가꾸기 위한 자기 관리  부안 민물낚시터 
840  임영웅   운동 취미  슬기로운 자가격리  유튜브로 팬과 소통
841  여름 실내운동  신논현 클라이밍장 클라이밍파크 존잼 취미운동
842  춘식이  춘식이는집순이  춘식이웹툰  요즘취미  춘식이캡쳐  운동  집순이의속마음
843  댄스배우기 언택트 취미생활  KPOP 방송댄스배우기 
844 코로나 취미 여러 가지 있어요 
845 부부요가 등록 완료  부부취미생활 저렴하게 하는법
846 취미 클래스에서 만나는 새로운 경험
847 성인취미   강동구검도   강동검도관에서 스트레스  면역력 높이기 
848 집콕 취미   틈나는 대로 블로그 글쓰기
849 부산댄스학원 서면댄스학원 서면제트엔댄스 수강료이벤트 중    직장인초보취미댄

In [10]:
import pandas as pd
from selenium import webdriver
driver = webdriver.Chrome('C:\chrome\chromedriver.exe')
import time

driver.get('http://www.naver.com')
time.sleep(2)

query_txt = '홈트'

#Step 2. 네이버 검색창에 "검색어" 검색
element = driver.find_element_by_id("query")
element.send_keys(query_txt)  # query_txt는 위에서 입력한 키워드

element.submit()
time.sleep(1)

# 'VIEW' 클릭
driver.find_element_by_link_text("VIEW").click( )
time.sleep(1)

# '블로그' 클릭
driver.find_element_by_link_text("블로그").click( )
time.sleep(1)

# '옵션' 클릭
driver.find_element_by_link_text("옵션").click( )

# 1년 선택
driver.find_element_by_css_selector('''#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.option > a:nth-child(8)''').click()

# 스크롤을 밑으로 내려주는 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 19431049)")
    time.sleep(1)

# n: 스크롤할 횟수 설정
n = 200 # 일종에 페이지수라고 보면 될 듯
i = 0
while i < n: # 이 조건이 만족되는 동안 반복 실행
    scroll_down(driver) # 스크롤 다운
    i = i+1
    
# 블로그 글 url들 수집
url_list = []
title_list = []

# URL_raw 크롤링 시작
articles = ".api_txt_lines.total_tit" # 중간에 띄어쓰기는 .으로 대체
article_raw = driver.find_elements_by_css_selector(articles)
article_raw[0]

# 크롤링한 url 정제 시작
for article in article_raw:
    url = article.get_attribute('href')   
    url_list.append(url)
    print(url)

# 1초 쉬어 주고,
time.sleep(1)
    
# 제목 크롤링 시작    
for article in article_raw:
    title = article.text
    title_list.append(title)
    print(title)

print("")
print('url갯수: ', len(url_list))
print('title갯수: ', len(title_list))
    
df = pd.DataFrame({'url':url_list, 'title':title_list})
df.to_excel("./naver/blog_exercise_url2.xlsx", index=False) #,engine='openpyxl'

https://blog.naver.com/qkrtmfdk52?Redirect=Log&logNo=222324005297
https://blog.naver.com/caucommando?Redirect=Log&logNo=222299396620
https://blog.naver.com/dkfnal2232?Redirect=Log&logNo=222336509837
https://blog.naver.com/74wonjin?Redirect=Log&logNo=222222759031
https://blog.naver.com/chance7665?Redirect=Log&logNo=222337931457
https://blog.naver.com/chulbal81?Redirect=Log&logNo=222345276647
https://blog.naver.com/jagaja1124?Redirect=Log&logNo=222318426810
https://blog.naver.com/redcham?Redirect=Log&logNo=222338998155
https://blog.naver.com/malahalo?Redirect=Log&logNo=222338572559
https://blog.naver.com/litsms?Redirect=Log&logNo=222344110993
https://blog.naver.com/miniavante?Redirect=Log&logNo=222335691102
https://blog.naver.com/dhgkssla?Redirect=Log&logNo=222232967588
https://blog.naver.com/gudwldnd?Redirect=Log&logNo=222294749670
https://blog.naver.com/yooxb1363?Redirect=Log&logNo=222341602206
https://blog.naver.com/dkfnal2232?Redirect=Log&logNo=222195360684
https://blog.naver.com/bi1

https://blog.naver.com/huhdong523?Redirect=Log&logNo=222317642432
https://blog.naver.com/chesungmin?Redirect=Log&logNo=222325354729
https://blog.naver.com/sjf7391?Redirect=Log&logNo=222326150944
https://blog.naver.com/chance7665?Redirect=Log&logNo=222181349336
https://blog.naver.com/summeryayi?Redirect=Log&logNo=222322845099
https://blog.naver.com/youplanner?Redirect=Log&logNo=222330098227
https://blog.naver.com/dmseod3537?Redirect=Log&logNo=222328960844
https://blog.naver.com/ehdqkd4559?Redirect=Log&logNo=222268895622
https://blog.naver.com/byki2013?Redirect=Log&logNo=222313806541
https://blog.naver.com/bna3643?Redirect=Log&logNo=222317723752
https://blog.naver.com/manhwapain?Redirect=Log&logNo=222320397786
https://blog.naver.com/happy_hworld?Redirect=Log&logNo=222319503965
https://blog.naver.com/kim385719?Redirect=Log&logNo=222320161828
https://sharehows.com/fitness_in_home
https://blog.naver.com/skcsy59?Redirect=Log&logNo=222322475470
https://blog.naver.com/888op?Redirect=Log&logNo=

https://blog.naver.com/sakurasake2?Redirect=Log&logNo=222306177531
https://blog.naver.com/alldaybunny?Redirect=Log&logNo=222291255211
https://blog.naver.com/chsj220?Redirect=Log&logNo=222307338731
https://blog.naver.com/psy7915?Redirect=Log&logNo=222169428990
https://blog.naver.com/yelin1890?Redirect=Log&logNo=222285722135
https://blog.naver.com/mel_holic?Redirect=Log&logNo=222308518422
https://blog.naver.com/luj3591?Redirect=Log&logNo=222291818714
https://blog.naver.com/boneedju?Redirect=Log&logNo=222326764003
https://blog.naver.com/14rang?Redirect=Log&logNo=222313116154
https://blog.naver.com/rinto0916?Redirect=Log&logNo=222295508578
https://blog.naver.com/bh2593?Redirect=Log&logNo=222287440704
https://ndolson.com/5496?category=715318
https://sharehows.com/home_training
https://blog.naver.com/yjewelry?Redirect=Log&logNo=222315306631
https://blog.naver.com/jsj_01?Redirect=Log&logNo=222279999084
https://blog.naver.com/uju_byul?Redirect=Log&logNo=222317291996
https://blog.naver.com/mina

https://blog.naver.com/etlandking?Redirect=Log&logNo=222280708547
https://blog.naver.com/woaini1914?Redirect=Log&logNo=222281181794
https://blog.naver.com/dhgkssla?Redirect=Log&logNo=222048760939
https://blog.naver.com/purefamily?Redirect=Log&logNo=222340554922
https://sharehows.com/hip-up-home-training
https://blog.naver.com/blooming0211?Redirect=Log&logNo=222277802432
https://blog.naver.com/sunflower_le?Redirect=Log&logNo=222334214239
https://blog.naver.com/luyan23666?Redirect=Log&logNo=222260543233
https://blog.naver.com/plm3334?Redirect=Log&logNo=222312748650
https://blog.naver.com/blue2489?Redirect=Log&logNo=222257918358
https://blog.naver.com/shop2930?Redirect=Log&logNo=222278146790
https://blog.naver.com/tlxmtlxm?Redirect=Log&logNo=222291525671
https://blog.naver.com/nailatti?Redirect=Log&logNo=222285816692
https://blog.naver.com/shinenany?Redirect=Log&logNo=222253077425
https://blog.naver.com/yennn73?Redirect=Log&logNo=222206820903
https://blog.naver.com/jua5084?Redirect=Log&lo

https://blog.naver.com/wjdgus1018?Redirect=Log&logNo=222300242973
https://blog.naver.com/jde7013?Redirect=Log&logNo=222252123327
https://blog.naver.com/tlxmtlxm?Redirect=Log&logNo=222259639473
https://blog.naver.com/11zoz11?Redirect=Log&logNo=222275088393
https://blog.naver.com/kchorong90?Redirect=Log&logNo=222252466316
https://blog.naver.com/neometeor?Redirect=Log&logNo=222229274785
https://blog.naver.com/chulbal81?Redirect=Log&logNo=222306609121
https://blog.naver.com/netjini1?Redirect=Log&logNo=222265113689
https://blog.naver.com/jeju8253?Redirect=Log&logNo=222264187184
https://blog.naver.com/p_jiwon0626?Redirect=Log&logNo=222250521016
https://blog.naver.com/gpdbs01?Redirect=Log&logNo=222257219985
https://blog.naver.com/songi00s?Redirect=Log&logNo=222320837985
https://blog.naver.com/ever331?Redirect=Log&logNo=222153394536
https://blog.naver.com/1030ysk?Redirect=Log&logNo=222100945495
https://blog.naver.com/jemu925?Redirect=Log&logNo=222277892971
https://blog.naver.com/jomail83?Redir

...

https://blog.naver.com/malahalo?Redirect=Log&logNo=222210719105
https://blog.naver.com/munyos?Redirect=Log&logNo=222215609556
https://blog.naver.com/8full?Redirect=Log&logNo=222184452238
https://blog.naver.com/jeong0207?Redirect=Log&logNo=222219266522
https://blog.naver.com/fmfaith?Redirect=Log&logNo=222211130768
https://blog.naver.com/ish185?Redirect=Log&logNo=222255477806
https://blog.naver.com/7tnals?Redirect=Log&logNo=222207611572
https://blog.naver.com/tjdghkdbwls?Redirect=Log&logNo=222295717003
https://blog.naver.com/eung1215?Redirect=Log&logNo=222132795081
https://blog.naver.com/duswk75?Redirect=Log&logNo=222202304036
https://blog.naver.com/sjy708?Redirect=Log&logNo=222252681313
https://blog.naver.com/magwi0903?Redirect=Log&logNo=222183732357
https://blog.naver.com/artknitting?Redirect=Log&logNo=222248474692
https://blog.naver.com/yooxb1363?Redirect=Log&logNo=222216661208
https://blog.naver.com/yeonjin85?Redirect=Log&logNo=222328716789
https://blog.naver.com/bingstyle?Redirect=L

https://blog.naver.com/marlang2?Redirect=Log&logNo=222011819985
https://blog.naver.com/sisipupu?Redirect=Log&logNo=222201929289
https://blog.naver.com/ch23052?Redirect=Log&logNo=222315442628
https://blog.naver.com/hoisulhi?Redirect=Log&logNo=222216006330
스텝퍼 소음 걱정없는 홈트
스텝퍼 재미있는 홈트 시간
전신운동 홈트로 하는 다이어트운동 추천 best3
유튜브 홈트 추천 받아서 시작했어요!
『낙성대 재활』거북목의 원인과 완화 하기 위한 홈트 방법!
셀프리쉬 저주파마사지기로 홈트 제대로 해보자!
홈트 운동기구, 유리한홈트로 끝!
홈트와 놀이매트로 사용 중인 지브라매트
명품 홈트매트 지브라매트는 확실히 다르다
엑스킹 로잉머신 Review (홈트,육아체력)
인바디체중계 홈트 필수템!
[운동매트/ 홈트매트 추천] 악마매트 저도 써봤어요!
무릎보호대 싹 풀려요!(등산 스쿼트 배드민턴 홈트)
캥거루타운 홈트밴드, 전신운동이 가능한 홈트기구
2021집에서 유산소운동 홈트 30초 루틴
무리한 ‘운동’ 강행? 제대로 된 ‘홈트’로 몸매 되살리기
팔꿈치통증파스 추천, 케토톱 붙이는통증전문가로 홈트 후 사용!
단백질 쉐이크 추천, 칼로바이 홈트 필수템 되겠네!
『낙성대PT』집에서 누구나 따라할 수 있는 홈트 - 복근운동
홈트매트 모아클래스 버닝매트 층간소음에서 해방되다
WPI보충제를 홈트에서도 프로틴쉐이크로 챙겨먹어요
AI 홈트레이닝 골프 and 다이어트 스마트홈트 솔직 리뷰
홈트 공간에 운동복, 보호대 장만은 티몬 여름미리준비
홈트 다이어트 쉽고 확실한 홈트레이닝 운동법
홈트 매트 솔티앤스웨티 대형 요가매트 예쁘고 실용적이야
나이키 푸쉬업바 그립 3.0 홈트 용품 AC4195-023 실사용기
홈트 덤벨 어깨운동 삼각근 데피니션 쇄골 만들기
요가삭스 논슬립 홈트양말 미끄럼방지 필

홈트 9일차,
홈트웨어, 스타일캠프 맨투맨 홈치마로 산책룩까지 예쁘게 완성
홈트 필수템 생수, 목넘김이 좋은 미네랄워터 퓨어수
유산소 운동(만보걷기) +근력운동(홈트) 7개월째
내 방이 헬스장! 방구석 홈트 입문 완전 정복
집에서 운동하기 전에 필수로 체크! 가정 내 홈트 주의사항
호스커스 요가양말 필라테스 미끄럼방지 롱삭스 홈트양말 추천
홈트 운동기구 라텍스 튜빙밴드 사용법 어깨운동 밴드
베어유 홈트 인강 여자근력운동 기초기본원리부터 세심하게
요즘하는 다이어트댄스 / 홈트 유튜브 추천
스타캣 여성운동복상의 홈트요가복
홈트중입니다♡4월1일-4월4일(실내자전거타기,만보걷기...
홈트의 시대! 여러분은 집에서 어떤 운동을 하시나요? 나만의 집...
아침 대용 선식, 홈트와 함께하는 다밀 밸런스
홈트 요가 추천 (4)셀프 두피 마사지
이크에크 음소거매트 홈트매트 추천
층간소음 없는 스핀 바이크, 실내 자전거로 홈트 스타투
다이어트 운동 유튜브 채널 홈트 추천!
미니멀리즘 홈트 기구 , 멜킨 정품 무게조절 덤벨 정답!
멜킨 실내자전거 홈트 강도별 운동 포인트
여자홈트, 내 집으로 찾아오는 다이어트 서비스를 만나보세요!
[끈기프로젝트]홈트따라하기 영상 추천
줌라이브수업으로 하는 홈트/하루 홈트
스마트체중계 홈트필수품 한가지!
홈트 추천 운태기 올뻔했는데 극뽁했다!
탄탄한 꿀벅지를 위하여! 탄탄한 다리 운동 홈트
내돈내산 바드라BADRA 홈트 요가매트 추천 :)
[홈트] 닌텐도 스위치 링피트 커스텀모드 설정하기
홈트 이고진 스텝퍼 구매후기 + 일주일 후기
여자 등운동 헬스장 & 홈트 원암덤벨로우 이렇게 하세요!
홈트매트 악마매트! (feat.건강한 형제들)
홈트할 때 바르면 좋은 웜업크림 추천!
허리스트레칭 칼스포츠 허리쭉쭉 홈트기구로도 좋아!
코로나 시국에 운동은 홈트! 영양은 파시코 리커버리팩!
넓은 요가매트 홈트에선 필수!
집순이를 위한 유투브 홈트 요가/필라테스 추천 #영어홈트...
하루 20분 홈트 40일 이후, 달라진 게 있을까?
90일 홈트 2기졸업(정선화 

홈트 운동기구 NEW 힘콩철봉
수, 목-홈트했슴돠
일상공감 돌려보게 :: 홈트로 추천해요!
fit 101 홈트운동기구의 정석! 피티쌤이 따로 필요 없는...
닌텐도 스위치 링피트 운동 한달홈트 후기(일반식 다이어트)
홈트 트레이닝 기구, 양치승 운동기구 피트 101로 코어근육...
홈트 매트 링피트 매트는 멜킨 그라운드 매트가 최고
홈트챌린지 42 - 살빼기운동 & 힙업운동
여자 덤벨운동 어깨부터 하체까지 홈트!
걷기 홈트 포스팅 멤버 구해요!! (1/11-1/23)
라텍스밴드 운동법 간편한 홈트 효과
재택근무 홈트레이닝 땅그부부 유튜브 홈트 추천
[홈트] 8mm 요가매트 TPE 와 2kg 덤벨, 멜킨 스포츠
선릉 올래곱창 / (또)브라이언스커피 스콘 / 홈트 스태퍼 일지!)
양치승 홈트운동기구 피트 101 홈쇼핑에서 완판된...
홈트로도 EMS운동 가능하다 리웨어스 솔직후기
락앤락 프로 쉐이커 밸런스로 홈트 보충제 먹기
삐약스핏 8일차-실내자전거, 홈트로빅, 하체스트레칭1
남자 여자 복근운동 루틴, 코로나 시대에 홈트가 필수!
요가매트 추천 나이팅핏 홈트매트 3번재 재구매 후기
케틀벨 홈트기구로 딱
홈트 도전! (여자 홈트레이닝 기구 보드30)
민쓰마켓 짐볼 사이즈 선택요령 및 홈트 짐볼운동법
홈트 필수템 코하루 에이피트 1평 스마트 훌라후프
가정용 로잉머신 노젓기 하며 홈트 전신운동 ⓦ헬스럽
<<자존감 UP 이미지홈트>> 수업리뷰 & 2기 개강안내 (5월 28일)
마미홈트 2월 셋째 주 리뷰
요술 EMS 버닝 다이어트 벨트, 여자 홈트기구 사용후기
홈트기구 뭅뭅 케틀벨 운동해요
새벽기상 / 독서 / 홈트
스텝퍼 홈트의 필수 알아보기
홈트 라텍스 힙 밴드로 매일 운동하는 딸
모아클래스 두꺼운 홈트매트 친환경요가매트 써본 후기
[유튜브]홈트추천 :: 티파니에서 에이핏까지
영국영어 어휘공부 | 교정영어홈트로 독학해 보아요...
이제 홈트는 필수! 여자 홈트레이닝 전신운동 루틴
[홈트 추천] 빌리부트캠프 유튜브 영상 따라한 후기
멜킨 노라인 줄넘기 / 훌라후프 개

온라인PT 없어져가는 복근 지키기 콰트 홈트 시작
요가소년으로 캐시홈트요가하고 공짜커피한잔
홈트 다이어트 멜킨 트램폴린 후기
홈트 한달 후기 마이다노랑 다노 눈뜨자마자 스트레칭
겨울방학 어린이 홈트로 딱! 키 크는 운동 함께 해요!
홈트 20201215-1216
홈트 후 스마트 인바디체중계로 자세히 체크하기
단백질보충제 VAT 옥타프로틴으로 홈트 시작
2020, D-34 / 다노 홈트 매트
일상공감 올인원 버터플라이 홈트용품추천
홈트 100일 프로젝트 완성
바이탈리스 요즘 홈트다이어트 중 프로바이오틱스로...
홈트, 다리가 예뻐지는?! 4분 하체운동
스콘 먹고 홈트하고 20201229-1230
홈트소도구 집에서 필라스틱으로 필라테스 해요:)
홈트운동기구 아령으로 상체운동(적당한 아령무게 찾기)
#집콕놀이 #코로나에 홈트가 짱 #미러볼 #가정용사이키
홈쇼핑 홈트운동기구 활용도 높은 멀티푸쉬업보드 피트101 후기
코어밸런스 홈트 매트, 푹신하니 좋아요♥
홈트로 하루 10분 운동 100일 성공
언더독스포츠 가정용철봉으로 홈트 턱걸이하기! 과연?!ㅋㅋㅋㅋ
홈트 3달 후기: 소미핏(미서원), 다노tv - 스트레칭 끝판왕
다이소 홈트용품 추천템 스마일~
21분으로 전신 체지방 불태우기, 홈트 합시다
코로나 블루극복!! 홈트 준비해보자
하루15분 홈트 하체운동루틴 4가지!(초보자 교육영상 有)
홈트 운동기구로 홈짐 2. 케틀벨 8kg, 12kg
아침공복운동 / 저녁홈트 / 복근 550개
[리뷰]다노샵 홈트 매트 사용후기
먹었으면 운동하자 EG홈트_음식과 운동의 특급 콜라보
21.2.19일 홈트!! 가슴운동
마이다노 온라인PT 홈트 마이다노토탈케어 1개월 수강후기
홈트&실내 사이클20210107-0108
아침요가 / 몸 풀기 / 홈트 (이 악물고ㅠ 천천히)
NBR 요가매트 폼롤러 필라테스 운동법 집에서 홈트로...
오!그래놀라 단백질 홈트 후 먹는 영양 가득 시리얼
주말이니 먹고 홈트하고 20201219-1220
달리기&홈트2주차 -공복유산소, 간헐적단식병행
홈트시작 : 토

In [15]:
import time
import re

start = time.time()

dict = {}    # 전체 크롤링 데이터를 담을 그릇

number = 1000  # 수집할 글 갯수 정하기

# 수집한 url 돌면서 데이터 수집
for i in tqdm(range(0, number)): # 끊긴 경우 여기 숫자만 변경하면 됨
    # 글 띄우기
    url = url_load['url'][i]
    driver = webdriver.Chrome('C:\chrome\chromedriver.exe')  # 윈도우는 "chromedriver.exe"
    driver.get(url)   # 글 띄우기
    
    # 크롤링
    
    try : 
        # iframe 접근
        driver.switch_to.frame('mainFrame')

        target_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성

        # 제목 크롤링 시작
        overlays = ".se-module.se-module-text.se-title-text"                                 
        tit = driver.find_element_by_css_selector(overlays)          # title
        title = tit.text
        title = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9| ]", " ", title)
        

        # 글쓴이 크롤링 시작
        overlays = ".nick"                                 
        nick = driver.find_element_by_css_selector(overlays)         # nickname
        nickname = nick.text

        # 날짜 크롤링
        overlays = ".se_publishDate.pcol2"                                 
        date = driver.find_element_by_css_selector(overlays)         # datetime
        datetime = date.text

        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"                                 
        contents = driver.find_elements_by_css_selector(overlays)    # contents

        content_list = []
        for content in contents:
            content_list.append(content.text)
             
        content_str = ' '.join(content_list) # 하나 이상의 공백을 치환           # content_str
        content_str = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9|.|,| ]", " ", content_str)
        
        # 글 하나는 target_info라는 딕셔너리에 담기게 되고,
        target_info['title'] = title
        target_info['nickname'] = nickname
        target_info['datetime'] = datetime
        target_info['content'] = content_str

        # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
        dict[i] = target_info
        time.sleep(1)
        
        # 크롤링이 성공하면 글 제목을 출력하게 되고,
        print(i, title)

        # 글 하나 크롤링 후 크롬 창을 닫습니다.
        driver.close()       
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동합니다.
    except:
        driver.close()
        time.sleep(1)
        continue
    
    # 중간,중간에 파일로 저장하기
    if i == 300 or 500 or 900 :
        # 판다스로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_excel("blog_exercise_content2.xlsx", encoding='utf-8-sig')
        time.sleep(3)

print('수집한 글 갯수: ', len(dict))
print(dict)

end = time.time() - start
print("used time: %dm%ds" % divmod(end,60))

NameError: name 'tqdm' is not defined

In [ ]:
from selenium import webdriver
driver = webdriver.Chrome('C:\chrome\chromedriver.exe')
import time

query_txt = '집에서 운동'

driver.get('http://www.naver.com')
time.sleep(2)

#Step 2. 네이버 검색창에 "검색어" 검색
element = driver.find_element_by_id("query")
element.send_keys(query_txt)  # query_txt는 위에서 입력한 키워드

element.submit()
time.sleep(1)

# 'VIEW' 클릭
driver.find_element_by_link_text("VIEW").click( )
time.sleep(1)

# '블로그' 클릭
driver.find_element_by_link_text("블로그").click( )
time.sleep(1)

# '옵션' 클릭
driver.find_element_by_link_text("옵션").click( )

# 1년 선택
driver.find_element_by_css_selector('''#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.option > a:nth-child(8)''').click()

# 스크롤을 밑으로 내려주는 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 19431049)")
    time.sleep(1)

# n: 스크롤할 횟수 설정
n = 200 # 일종에 페이지수라고 보면 될 듯
i = 0
while i < n: # 이 조건이 만족되는 동안 반복 실행
    scroll_down(driver) # 스크롤 다운
    i = i+1
    
# 블로그 글 url들 수집
url_list = []
title_list = []

# URL_raw 크롤링 시작
articles = ".api_txt_lines.total_tit" # 중간에 띄어쓰기는 .으로 대체
article_raw = driver.find_elements_by_css_selector(articles)
article_raw[0]

# 크롤링한 url 정제 시작
for article in article_raw:
    url = article.get_attribute('href')   
    url_list.append(url)
    print(url)

# 1초 쉬어 주고,
time.sleep(1)
    
# 제목 크롤링 시작    
for article in article_raw:
    title = article.text
    title_list.append(title)
    print(title)

print("")
print('url갯수: ', len(url_list))
print('title갯수: ', len(title_list))
    
df = pd.DataFrame({'url':url_list, 'title':title_list})
df.to_excel("./naver/blog_exercise_url3.xlsx", index=False) #,engine='openpyxl'

In [ ]:
url_load = pd.read_excel("./naver/blog_exercise_url2.xlsx",engine='openpyxl')

num_list = len(url_load)

print(num_list)
url_load

import time
import re

start = time.time()

dict = {}    # 전체 크롤링 데이터를 담을 그릇

number = 1000  # 수집할 글 갯수 정하기

# 수집한 url 돌면서 데이터 수집
for i in tqdm(range(0, number)): # 끊긴 경우 여기 숫자만 변경하면 됨
    # 글 띄우기
    url = url_load['url'][i]
    driver = webdriver.Chrome('C:\chrome\chromedriver.exe')  # 윈도우는 "chromedriver.exe"
    driver.get(url)   # 글 띄우기
    
    # 크롤링
    
    try : 
        # iframe 접근
        driver.switch_to.frame('mainFrame')

        target_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성

        # 제목 크롤링 시작
        overlays = ".se-module.se-module-text.se-title-text"                                 
        tit = driver.find_element_by_css_selector(overlays)          # title
        title = tit.text
        title = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9| ]", " ", title)
        

        # 글쓴이 크롤링 시작
        overlays = ".nick"                                 
        nick = driver.find_element_by_css_selector(overlays)         # nickname
        nickname = nick.text

        # 날짜 크롤링
        overlays = ".se_publishDate.pcol2"                                 
        date = driver.find_element_by_css_selector(overlays)         # datetime
        datetime = date.text

        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"                                 
        contents = driver.find_elements_by_css_selector(overlays)    # contents

        content_list = []
        for content in contents:
            content_list.append(content.text)
             
        content_str = ' '.join(content_list) # 하나 이상의 공백을 치환           # content_str
        content_str = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9|.|,| ]", " ", content_str)
        
        # 글 하나는 target_info라는 딕셔너리에 담기게 되고,
        target_info['title'] = title
        target_info['nickname'] = nickname
        target_info['datetime'] = datetime
        target_info['content'] = content_str

        # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
        dict[i] = target_info
        time.sleep(1)
        
        # 크롤링이 성공하면 글 제목을 출력하게 되고,
        print(i, title)

        # 글 하나 크롤링 후 크롬 창을 닫습니다.
        driver.close()       
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동합니다.
    except:
        driver.close()
        time.sleep(1)
        continue
    
    # 중간,중간에 파일로 저장하기
    if i == 100 or 300 or 500 :
        # 판다스로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_excel("blog_exercise_content3.xlsx", encoding='utf-8-sig')
        time.sleep(3)

print('수집한 글 갯수: ', len(dict))
print(dict)

end = time.time() - start
print("used time: %dm%ds" % divmod(end,60))

In [ ]:
from selenium import webdriver
driver = webdriver.Chrome('C:\chrome\chromedriver.exe')
import time

query_txt = '헬스'

driver.get('http://www.naver.com')
time.sleep(2)

#Step 2. 네이버 검색창에 "검색어" 검색
element = driver.find_element_by_id("query")
element.send_keys(query_txt)  # query_txt는 위에서 입력한 키워드

element.submit()
time.sleep(1)

# 'VIEW' 클릭
driver.find_element_by_link_text("VIEW").click( )
time.sleep(1)

# '블로그' 클릭
driver.find_element_by_link_text("블로그").click( )
time.sleep(1)

# '옵션' 클릭
driver.find_element_by_link_text("옵션").click( )

# 1년 선택
driver.find_element_by_css_selector('''#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.option > a:nth-child(8)''').click()

# 스크롤을 밑으로 내려주는 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 19431049)")
    time.sleep(1)

# n: 스크롤할 횟수 설정
n = 200 # 일종에 페이지수라고 보면 될 듯
i = 0
while i < n: # 이 조건이 만족되는 동안 반복 실행
    scroll_down(driver) # 스크롤 다운
    i = i+1
    
# 블로그 글 url들 수집
url_list = []
title_list = []

# URL_raw 크롤링 시작
articles = ".api_txt_lines.total_tit" # 중간에 띄어쓰기는 .으로 대체
article_raw = driver.find_elements_by_css_selector(articles)
article_raw[0]

# 크롤링한 url 정제 시작
for article in article_raw:
    url = article.get_attribute('href')   
    url_list.append(url)
    print(url)

# 1초 쉬어 주고,
time.sleep(1)
    
# 제목 크롤링 시작    
for article in article_raw:
    title = article.text
    title_list.append(title)
    print(title)

print("")
print('url갯수: ', len(url_list))
print('title갯수: ', len(title_list))
    
df = pd.DataFrame({'url':url_list, 'title':title_list})
df.to_excel("./naver/blog_exercise_url4.xlsx", index=False) #,engine='openpyxl'

In [ ]:
url_load = pd.read_excel("./naver/blog_exercise_url2.xlsx",engine='openpyxl')

num_list = len(url_load)

print(num_list)
url_load

import time
import re

start = time.time()

dict = {}    # 전체 크롤링 데이터를 담을 그릇

number = 1000  # 수집할 글 갯수 정하기

# 수집한 url 돌면서 데이터 수집
for i in tqdm(range(0, number)): # 끊긴 경우 여기 숫자만 변경하면 됨
    # 글 띄우기
    url = url_load['url'][i]
    driver = webdriver.Chrome('C:\chrome\chromedriver.exe')  # 윈도우는 "chromedriver.exe"
    driver.get(url)   # 글 띄우기
    
    # 크롤링
    
    try : 
        # iframe 접근
        driver.switch_to.frame('mainFrame')

        target_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성

        # 제목 크롤링 시작
        overlays = ".se-module.se-module-text.se-title-text"                                 
        tit = driver.find_element_by_css_selector(overlays)          # title
        title = tit.text
        title = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9| ]", " ", title)
        

        # 글쓴이 크롤링 시작
        overlays = ".nick"                                 
        nick = driver.find_element_by_css_selector(overlays)         # nickname
        nickname = nick.text

        # 날짜 크롤링
        overlays = ".se_publishDate.pcol2"                                 
        date = driver.find_element_by_css_selector(overlays)         # datetime
        datetime = date.text

        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"                                 
        contents = driver.find_elements_by_css_selector(overlays)    # contents

        content_list = []
        for content in contents:
            content_list.append(content.text)
             
        content_str = ' '.join(content_list) # 하나 이상의 공백을 치환           # content_str
        content_str = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9|.|,| ]", " ", content_str)
        
        # 글 하나는 target_info라는 딕셔너리에 담기게 되고,
        target_info['title'] = title
        target_info['nickname'] = nickname
        target_info['datetime'] = datetime
        target_info['content'] = content_str

        # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
        dict[i] = target_info
        time.sleep(1)
        
        # 크롤링이 성공하면 글 제목을 출력하게 되고,
        print(i, title)

        # 글 하나 크롤링 후 크롬 창을 닫습니다.
        driver.close()       
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동합니다.
    except:
        driver.close()
        time.sleep(1)
        continue
    
    # 중간,중간에 파일로 저장하기
    if i == 100 or 300 or 500 :
        # 판다스로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_excel("blog_exercise_content4.xlsx", encoding='utf-8-sig')
        time.sleep(3)

print('수집한 글 갯수: ', len(dict))
print(dict)

end = time.time() - start
print("used time: %dm%ds" % divmod(end,60))

In [ ]:
from selenium import webdriver
driver = webdriver.Chrome('C:\chrome\chromedriver.exe')
import time

query_txt = '다이어트'

driver.get('http://www.naver.com')
time.sleep(2)

#Step 2. 네이버 검색창에 "검색어" 검색
element = driver.find_element_by_id("query")
element.send_keys(query_txt)  # query_txt는 위에서 입력한 키워드

element.submit()
time.sleep(1)

# 'VIEW' 클릭
driver.find_element_by_link_text("VIEW").click( )
time.sleep(1)

# '블로그' 클릭
driver.find_element_by_link_text("블로그").click( )
time.sleep(1)

# '옵션' 클릭
driver.find_element_by_link_text("옵션").click( )

# 1년 선택
driver.find_element_by_css_selector('''#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.option > a:nth-child(8)''').click()

# 스크롤을 밑으로 내려주는 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 19431049)")
    time.sleep(1)

# n: 스크롤할 횟수 설정
n = 200 # 일종에 페이지수라고 보면 될 듯
i = 0
while i < n: # 이 조건이 만족되는 동안 반복 실행
    scroll_down(driver) # 스크롤 다운
    i = i+1
    
# 블로그 글 url들 수집
url_list = []
title_list = []

# URL_raw 크롤링 시작
articles = ".api_txt_lines.total_tit" # 중간에 띄어쓰기는 .으로 대체
article_raw = driver.find_elements_by_css_selector(articles)
article_raw[0]

# 크롤링한 url 정제 시작
for article in article_raw:
    url = article.get_attribute('href')   
    url_list.append(url)
    print(url)

# 1초 쉬어 주고,
time.sleep(1)
    
# 제목 크롤링 시작    
for article in article_raw:
    title = article.text
    title_list.append(title)
    print(title)

print("")
print('url갯수: ', len(url_list))
print('title갯수: ', len(title_list))
    
df = pd.DataFrame({'url':url_list, 'title':title_list})
df.to_excel("./naver/blog_exercise_url5.xlsx", index=False) #,engine='openpyxl'

In [ ]:
url_load = pd.read_excel("./naver/blog_exercise_url2.xlsx",engine='openpyxl')

num_list = len(url_load)

print(num_list)
url_load

import time
import re

start = time.time()

dict = {}    # 전체 크롤링 데이터를 담을 그릇

number = 1000  # 수집할 글 갯수 정하기

# 수집한 url 돌면서 데이터 수집
for i in tqdm(range(0, number)): # 끊긴 경우 여기 숫자만 변경하면 됨
    # 글 띄우기
    url = url_load['url'][i]
    driver = webdriver.Chrome('C:\chrome\chromedriver.exe')  # 윈도우는 "chromedriver.exe"
    driver.get(url)   # 글 띄우기
    
    # 크롤링
    
    try : 
        # iframe 접근
        driver.switch_to.frame('mainFrame')

        target_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성

        # 제목 크롤링 시작
        overlays = ".se-module.se-module-text.se-title-text"                                 
        tit = driver.find_element_by_css_selector(overlays)          # title
        title = tit.text
        title = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9| ]", " ", title)
        

        # 글쓴이 크롤링 시작
        overlays = ".nick"                                 
        nick = driver.find_element_by_css_selector(overlays)         # nickname
        nickname = nick.text

        # 날짜 크롤링
        overlays = ".se_publishDate.pcol2"                                 
        date = driver.find_element_by_css_selector(overlays)         # datetime
        datetime = date.text

        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"                                 
        contents = driver.find_elements_by_css_selector(overlays)    # contents

        content_list = []
        for content in contents:
            content_list.append(content.text)
             
        content_str = ' '.join(content_list) # 하나 이상의 공백을 치환           # content_str
        content_str = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9|.|,| ]", " ", content_str)
        
        # 글 하나는 target_info라는 딕셔너리에 담기게 되고,
        target_info['title'] = title
        target_info['nickname'] = nickname
        target_info['datetime'] = datetime
        target_info['content'] = content_str

        # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
        dict[i] = target_info
        time.sleep(1)
        
        # 크롤링이 성공하면 글 제목을 출력하게 되고,
        print(i, title)

        # 글 하나 크롤링 후 크롬 창을 닫습니다.
        driver.close()       
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동합니다.
    except:
        driver.close()
        time.sleep(1)
        continue
    
    # 중간,중간에 파일로 저장하기
    if i == 100 or 300 or 500 :
        # 판다스로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_excel("blog_exercise_content5.xlsx", encoding='utf-8-sig')
        time.sleep(3)

print('수집한 글 갯수: ', len(dict))
print(dict)

end = time.time() - start
print("used time: %dm%ds" % divmod(end,60))

In [ ]:
from selenium import webdriver
driver = webdriver.Chrome('C:\chrome\chromedriver.exe')
import time

query_txt = '케틀벨'

driver.get('http://www.naver.com')
time.sleep(2)

#Step 2. 네이버 검색창에 "검색어" 검색
element = driver.find_element_by_id("query")
element.send_keys(query_txt)  # query_txt는 위에서 입력한 키워드

element.submit()
time.sleep(1)

# 'VIEW' 클릭
driver.find_element_by_link_text("VIEW").click( )
time.sleep(1)

# '블로그' 클릭
driver.find_element_by_link_text("블로그").click( )
time.sleep(1)

# '옵션' 클릭
driver.find_element_by_link_text("옵션").click( )

# 1년 선택
driver.find_element_by_css_selector('''#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.option > a:nth-child(8)''').click()

# 스크롤을 밑으로 내려주는 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 19431049)")
    time.sleep(1)

# n: 스크롤할 횟수 설정
n = 200 # 일종에 페이지수라고 보면 될 듯
i = 0
while i < n: # 이 조건이 만족되는 동안 반복 실행
    scroll_down(driver) # 스크롤 다운
    i = i+1
    
# 블로그 글 url들 수집
url_list = []
title_list = []

# URL_raw 크롤링 시작
articles = ".api_txt_lines.total_tit" # 중간에 띄어쓰기는 .으로 대체
article_raw = driver.find_elements_by_css_selector(articles)
article_raw[0]

# 크롤링한 url 정제 시작
for article in article_raw:
    url = article.get_attribute('href')   
    url_list.append(url)
    print(url)

# 1초 쉬어 주고,
time.sleep(1)
    
# 제목 크롤링 시작    
for article in article_raw:
    title = article.text
    title_list.append(title)
    print(title)

print("")
print('url갯수: ', len(url_list))
print('title갯수: ', len(title_list))
    
df = pd.DataFrame({'url':url_list, 'title':title_list})
df.to_excel("./naver/blog_exercise_url6.xlsx", index=False) #,engine='openpyxl'

In [ ]:
url_load = pd.read_excel("./naver/blog_exercise_url2.xlsx",engine='openpyxl')

num_list = len(url_load)

print(num_list)
url_load

import time
import re

start = time.time()

dict = {}    # 전체 크롤링 데이터를 담을 그릇

number = 1000  # 수집할 글 갯수 정하기

# 수집한 url 돌면서 데이터 수집
for i in tqdm(range(0, number)): # 끊긴 경우 여기 숫자만 변경하면 됨
    # 글 띄우기
    url = url_load['url'][i]
    driver = webdriver.Chrome('C:\chrome\chromedriver.exe')  # 윈도우는 "chromedriver.exe"
    driver.get(url)   # 글 띄우기
    
    # 크롤링
    
    try : 
        # iframe 접근
        driver.switch_to.frame('mainFrame')

        target_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성

        # 제목 크롤링 시작
        overlays = ".se-module.se-module-text.se-title-text"                                 
        tit = driver.find_element_by_css_selector(overlays)          # title
        title = tit.text
        title = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9| ]", " ", title)
        

        # 글쓴이 크롤링 시작
        overlays = ".nick"                                 
        nick = driver.find_element_by_css_selector(overlays)         # nickname
        nickname = nick.text

        # 날짜 크롤링
        overlays = ".se_publishDate.pcol2"                                 
        date = driver.find_element_by_css_selector(overlays)         # datetime
        datetime = date.text

        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"                                 
        contents = driver.find_elements_by_css_selector(overlays)    # contents

        content_list = []
        for content in contents:
            content_list.append(content.text)
             
        content_str = ' '.join(content_list) # 하나 이상의 공백을 치환           # content_str
        content_str = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9|.|,| ]", " ", content_str)
        
        # 글 하나는 target_info라는 딕셔너리에 담기게 되고,
        target_info['title'] = title
        target_info['nickname'] = nickname
        target_info['datetime'] = datetime
        target_info['content'] = content_str

        # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
        dict[i] = target_info
        time.sleep(1)
        
        # 크롤링이 성공하면 글 제목을 출력하게 되고,
        print(i, title)

        # 글 하나 크롤링 후 크롬 창을 닫습니다.
        driver.close()       
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동합니다.
    except:
        driver.close()
        time.sleep(1)
        continue
    
    # 중간,중간에 파일로 저장하기
    if i == 100 or 300 or 500 :
        # 판다스로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_excel("blog_exercise_content6.xlsx", encoding='utf-8-sig')
        time.sleep(3)

print('수집한 글 갯수: ', len(dict))
print(dict)

end = time.time() - start
print("used time: %dm%ds" % divmod(end,60))

In [ ]:
from selenium import webdriver
driver = webdriver.Chrome('C:\chrome\chromedriver.exe')
import time

query_txt = '폼롤러'

driver.get('http://www.naver.com')
time.sleep(2)

#Step 2. 네이버 검색창에 "검색어" 검색
element = driver.find_element_by_id("query")
element.send_keys(query_txt)  # query_txt는 위에서 입력한 키워드

element.submit()
time.sleep(1)

# 'VIEW' 클릭
driver.find_element_by_link_text("VIEW").click( )
time.sleep(1)

# '블로그' 클릭
driver.find_element_by_link_text("블로그").click( )
time.sleep(1)

# '옵션' 클릭
driver.find_element_by_link_text("옵션").click( )

# 1년 선택
driver.find_element_by_css_selector('''#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.option > a:nth-child(8)''').click()

# 스크롤을 밑으로 내려주는 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 19431049)")
    time.sleep(1)

# n: 스크롤할 횟수 설정
n = 200 # 일종에 페이지수라고 보면 될 듯
i = 0
while i < n: # 이 조건이 만족되는 동안 반복 실행
    scroll_down(driver) # 스크롤 다운
    i = i+1
    
# 블로그 글 url들 수집
url_list = []
title_list = []

# URL_raw 크롤링 시작
articles = ".api_txt_lines.total_tit" # 중간에 띄어쓰기는 .으로 대체
article_raw = driver.find_elements_by_css_selector(articles)
article_raw[0]

# 크롤링한 url 정제 시작
for article in article_raw:
    url = article.get_attribute('href')   
    url_list.append(url)
    print(url)

# 1초 쉬어 주고,
time.sleep(1)
    
# 제목 크롤링 시작    
for article in article_raw:
    title = article.text
    title_list.append(title)
    print(title)

print("")
print('url갯수: ', len(url_list))
print('title갯수: ', len(title_list))
    
df = pd.DataFrame({'url':url_list, 'title':title_list})
df.to_excel("./naver/blog_exercise_url7.xlsx", index=False) #,engine='openpyxl'

In [ ]:
url_load = pd.read_excel("./naver/blog_exercise_url2.xlsx",engine='openpyxl')

num_list = len(url_load)

print(num_list)
url_load

import time
import re

start = time.time()

dict = {}    # 전체 크롤링 데이터를 담을 그릇

number = 1000  # 수집할 글 갯수 정하기

# 수집한 url 돌면서 데이터 수집
for i in tqdm(range(0, number)): # 끊긴 경우 여기 숫자만 변경하면 됨
    # 글 띄우기
    url = url_load['url'][i]
    driver = webdriver.Chrome('C:\chrome\chromedriver.exe')  # 윈도우는 "chromedriver.exe"
    driver.get(url)   # 글 띄우기
    
    # 크롤링
    
    try : 
        # iframe 접근
        driver.switch_to.frame('mainFrame')

        target_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성

        # 제목 크롤링 시작
        overlays = ".se-module.se-module-text.se-title-text"                                 
        tit = driver.find_element_by_css_selector(overlays)          # title
        title = tit.text
        title = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9| ]", " ", title)
        

        # 글쓴이 크롤링 시작
        overlays = ".nick"                                 
        nick = driver.find_element_by_css_selector(overlays)         # nickname
        nickname = nick.text

        # 날짜 크롤링
        overlays = ".se_publishDate.pcol2"                                 
        date = driver.find_element_by_css_selector(overlays)         # datetime
        datetime = date.text

        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"                                 
        contents = driver.find_elements_by_css_selector(overlays)    # contents

        content_list = []
        for content in contents:
            content_list.append(content.text)
             
        content_str = ' '.join(content_list) # 하나 이상의 공백을 치환           # content_str
        content_str = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9|.|,| ]", " ", content_str)
        
        # 글 하나는 target_info라는 딕셔너리에 담기게 되고,
        target_info['title'] = title
        target_info['nickname'] = nickname
        target_info['datetime'] = datetime
        target_info['content'] = content_str

        # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
        dict[i] = target_info
        time.sleep(1)
        
        # 크롤링이 성공하면 글 제목을 출력하게 되고,
        print(i, title)

        # 글 하나 크롤링 후 크롬 창을 닫습니다.
        driver.close()       
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동합니다.
    except:
        driver.close()
        time.sleep(1)
        continue
    
    # 중간,중간에 파일로 저장하기
    if i == 100 or 300 or 500 :
        # 판다스로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_excel("blog_exercise_content7.xlsx", encoding='utf-8-sig')
        time.sleep(3)

print('수집한 글 갯수: ', len(dict))
print(dict)

end = time.time() - start
print("used time: %dm%ds" % divmod(end,60))

In [ ]:
from selenium import webdriver
driver = webdriver.Chrome('C:\chrome\chromedriver.exe')
import time

query_txt = '덤벨'

driver.get('http://www.naver.com')
time.sleep(2)

#Step 2. 네이버 검색창에 "검색어" 검색
element = driver.find_element_by_id("query")
element.send_keys(query_txt)  # query_txt는 위에서 입력한 키워드

element.submit()
time.sleep(1)

# 'VIEW' 클릭
driver.find_element_by_link_text("VIEW").click( )
time.sleep(1)

# '블로그' 클릭
driver.find_element_by_link_text("블로그").click( )
time.sleep(1)

# '옵션' 클릭
driver.find_element_by_link_text("옵션").click( )

# 1년 선택
driver.find_element_by_css_selector('''#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.option > a:nth-child(8)''').click()

# 스크롤을 밑으로 내려주는 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 19431049)")
    time.sleep(1)

# n: 스크롤할 횟수 설정
n = 200 # 일종에 페이지수라고 보면 될 듯
i = 0
while i < n: # 이 조건이 만족되는 동안 반복 실행
    scroll_down(driver) # 스크롤 다운
    i = i+1
    
# 블로그 글 url들 수집
url_list = []
title_list = []

# URL_raw 크롤링 시작
articles = ".api_txt_lines.total_tit" # 중간에 띄어쓰기는 .으로 대체
article_raw = driver.find_elements_by_css_selector(articles)
article_raw[0]

# 크롤링한 url 정제 시작
for article in article_raw:
    url = article.get_attribute('href')   
    url_list.append(url)
    print(url)

# 1초 쉬어 주고,
time.sleep(1)
    
# 제목 크롤링 시작    
for article in article_raw:
    title = article.text
    title_list.append(title)
    print(title)

print("")
print('url갯수: ', len(url_list))
print('title갯수: ', len(title_list))
    
df = pd.DataFrame({'url':url_list, 'title':title_list})
df.to_excel("./naver/blog_exercise_url8.xlsx", index=False) #,engine='openpyxl'

In [ ]:
url_load = pd.read_excel("./naver/blog_exercise_url2.xlsx",engine='openpyxl')

num_list = len(url_load)

print(num_list)
url_load

import time
import re

start = time.time()

dict = {}    # 전체 크롤링 데이터를 담을 그릇

number = 1000  # 수집할 글 갯수 정하기

# 수집한 url 돌면서 데이터 수집
for i in tqdm(range(0, number)): # 끊긴 경우 여기 숫자만 변경하면 됨
    # 글 띄우기
    url = url_load['url'][i]
    driver = webdriver.Chrome('C:\chrome\chromedriver.exe')  # 윈도우는 "chromedriver.exe"
    driver.get(url)   # 글 띄우기
    
    # 크롤링
    
    try : 
        # iframe 접근
        driver.switch_to.frame('mainFrame')

        target_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성

        # 제목 크롤링 시작
        overlays = ".se-module.se-module-text.se-title-text"                                 
        tit = driver.find_element_by_css_selector(overlays)          # title
        title = tit.text
        title = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9| ]", " ", title)
        

        # 글쓴이 크롤링 시작
        overlays = ".nick"                                 
        nick = driver.find_element_by_css_selector(overlays)         # nickname
        nickname = nick.text

        # 날짜 크롤링
        overlays = ".se_publishDate.pcol2"                                 
        date = driver.find_element_by_css_selector(overlays)         # datetime
        datetime = date.text

        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"                                 
        contents = driver.find_elements_by_css_selector(overlays)    # contents

        content_list = []
        for content in contents:
            content_list.append(content.text)
             
        content_str = ' '.join(content_list) # 하나 이상의 공백을 치환           # content_str
        content_str = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9|.|,| ]", " ", content_str)
        
        # 글 하나는 target_info라는 딕셔너리에 담기게 되고,
        target_info['title'] = title
        target_info['nickname'] = nickname
        target_info['datetime'] = datetime
        target_info['content'] = content_str

        # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
        dict[i] = target_info
        time.sleep(1)
        
        # 크롤링이 성공하면 글 제목을 출력하게 되고,
        print(i, title)

        # 글 하나 크롤링 후 크롬 창을 닫습니다.
        driver.close()       
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동합니다.
    except:
        driver.close()
        time.sleep(1)
        continue
    
    # 중간,중간에 파일로 저장하기
    if i == 100 or 300 or 500 :
        # 판다스로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_excel("blog_exercise_content8.xlsx", encoding='utf-8-sig')
        time.sleep(3)

print('수집한 글 갯수: ', len(dict))
print(dict)

end = time.time() - start
print("used time: %dm%ds" % divmod(end,60))

In [ ]:
from selenium import webdriver
driver = webdriver.Chrome('C:\chrome\chromedriver.exe')
import time

query_txt = '아령'

driver.get('http://www.naver.com')
time.sleep(2)

#Step 2. 네이버 검색창에 "검색어" 검색
element = driver.find_element_by_id("query")
element.send_keys(query_txt)  # query_txt는 위에서 입력한 키워드

element.submit()
time.sleep(1)

# 'VIEW' 클릭
driver.find_element_by_link_text("VIEW").click( )
time.sleep(1)

# '블로그' 클릭
driver.find_element_by_link_text("블로그").click( )
time.sleep(1)

# '옵션' 클릭
driver.find_element_by_link_text("옵션").click( )

# 1년 선택
driver.find_element_by_css_selector('''#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.option > a:nth-child(8)''').click()

# 스크롤을 밑으로 내려주는 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 19431049)")
    time.sleep(1)

# n: 스크롤할 횟수 설정
n = 200 # 일종에 페이지수라고 보면 될 듯
i = 0
while i < n: # 이 조건이 만족되는 동안 반복 실행
    scroll_down(driver) # 스크롤 다운
    i = i+1
    
# 블로그 글 url들 수집
url_list = []
title_list = []

# URL_raw 크롤링 시작
articles = ".api_txt_lines.total_tit" # 중간에 띄어쓰기는 .으로 대체
article_raw = driver.find_elements_by_css_selector(articles)
article_raw[0]

# 크롤링한 url 정제 시작
for article in article_raw:
    url = article.get_attribute('href')   
    url_list.append(url)
    print(url)

# 1초 쉬어 주고,
time.sleep(1)
    
# 제목 크롤링 시작    
for article in article_raw:
    title = article.text
    title_list.append(title)
    print(title)

print("")
print('url갯수: ', len(url_list))
print('title갯수: ', len(title_list))
    
df = pd.DataFrame({'url':url_list, 'title':title_list})
df.to_excel("./naver/blog_exercise_url9.xlsx", index=False) #,engine='openpyxl'

In [ ]:
url_load = pd.read_excel("./naver/blog_exercise_url2.xlsx",engine='openpyxl')

num_list = len(url_load)

print(num_list)
url_load

import time
import re

start = time.time()

dict = {}    # 전체 크롤링 데이터를 담을 그릇

number = 1000  # 수집할 글 갯수 정하기

# 수집한 url 돌면서 데이터 수집
for i in tqdm(range(0, number)): # 끊긴 경우 여기 숫자만 변경하면 됨
    # 글 띄우기
    url = url_load['url'][i]
    driver = webdriver.Chrome('C:\chrome\chromedriver.exe')  # 윈도우는 "chromedriver.exe"
    driver.get(url)   # 글 띄우기
    
    # 크롤링
    
    try : 
        # iframe 접근
        driver.switch_to.frame('mainFrame')

        target_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성

        # 제목 크롤링 시작
        overlays = ".se-module.se-module-text.se-title-text"                                 
        tit = driver.find_element_by_css_selector(overlays)          # title
        title = tit.text
        title = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9| ]", " ", title)
        

        # 글쓴이 크롤링 시작
        overlays = ".nick"                                 
        nick = driver.find_element_by_css_selector(overlays)         # nickname
        nickname = nick.text

        # 날짜 크롤링
        overlays = ".se_publishDate.pcol2"                                 
        date = driver.find_element_by_css_selector(overlays)         # datetime
        datetime = date.text

        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"                                 
        contents = driver.find_elements_by_css_selector(overlays)    # contents

        content_list = []
        for content in contents:
            content_list.append(content.text)
             
        content_str = ' '.join(content_list) # 하나 이상의 공백을 치환           # content_str
        content_str = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9|.|,| ]", " ", content_str)
        
        # 글 하나는 target_info라는 딕셔너리에 담기게 되고,
        target_info['title'] = title
        target_info['nickname'] = nickname
        target_info['datetime'] = datetime
        target_info['content'] = content_str

        # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
        dict[i] = target_info
        time.sleep(1)
        
        # 크롤링이 성공하면 글 제목을 출력하게 되고,
        print(i, title)

        # 글 하나 크롤링 후 크롬 창을 닫습니다.
        driver.close()       
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동합니다.
    except:
        driver.close()
        time.sleep(1)
        continue
    
    # 중간,중간에 파일로 저장하기
    if i == 100 or 300 or 500 :
        # 판다스로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_excel("blog_exercise_content9.xlsx", encoding='utf-8-sig')
        time.sleep(3)

print('수집한 글 갯수: ', len(dict))
print(dict)

end = time.time() - start
print("used time: %dm%ds" % divmod(end,60))

In [ ]:
from selenium import webdriver
driver = webdriver.Chrome('C:\chrome\chromedriver.exe')
import time

query_txt = '스쿼드'

driver.get('http://www.naver.com')
time.sleep(2)

#Step 2. 네이버 검색창에 "검색어" 검색
element = driver.find_element_by_id("query")
element.send_keys(query_txt)  # query_txt는 위에서 입력한 키워드

element.submit()
time.sleep(1)

# 'VIEW' 클릭
driver.find_element_by_link_text("VIEW").click( )
time.sleep(1)

# '블로그' 클릭
driver.find_element_by_link_text("블로그").click( )
time.sleep(1)

# '옵션' 클릭
driver.find_element_by_link_text("옵션").click( )

# 1년 선택
driver.find_element_by_css_selector('''#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.option > a:nth-child(8)''').click()

# 스크롤을 밑으로 내려주는 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 19431049)")
    time.sleep(1)

# n: 스크롤할 횟수 설정
n = 200 # 일종에 페이지수라고 보면 될 듯
i = 0
while i < n: # 이 조건이 만족되는 동안 반복 실행
    scroll_down(driver) # 스크롤 다운
    i = i+1
    
# 블로그 글 url들 수집
url_list = []
title_list = []

# URL_raw 크롤링 시작
articles = ".api_txt_lines.total_tit" # 중간에 띄어쓰기는 .으로 대체
article_raw = driver.find_elements_by_css_selector(articles)
article_raw[0]

# 크롤링한 url 정제 시작
for article in article_raw:
    url = article.get_attribute('href')   
    url_list.append(url)
    print(url)

# 1초 쉬어 주고,
time.sleep(1)
    
# 제목 크롤링 시작    
for article in article_raw:
    title = article.text
    title_list.append(title)
    print(title)

print("")
print('url갯수: ', len(url_list))
print('title갯수: ', len(title_list))
    
df = pd.DataFrame({'url':url_list, 'title':title_list})
df.to_excel("./naver/blog_exercise_url10.xlsx", index=False) #,engine='openpyxl'

In [ ]:
url_load = pd.read_excel("./naver/blog_exercise_url2.xlsx",engine='openpyxl')

num_list = len(url_load)

print(num_list)
url_load

import time
import re

start = time.time()

dict = {}    # 전체 크롤링 데이터를 담을 그릇

number = 1000  # 수집할 글 갯수 정하기

# 수집한 url 돌면서 데이터 수집
for i in tqdm(range(0, number)): # 끊긴 경우 여기 숫자만 변경하면 됨
    # 글 띄우기
    url = url_load['url'][i]
    driver = webdriver.Chrome('C:\chrome\chromedriver.exe')  # 윈도우는 "chromedriver.exe"
    driver.get(url)   # 글 띄우기
    
    # 크롤링
    
    try : 
        # iframe 접근
        driver.switch_to.frame('mainFrame')

        target_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성

        # 제목 크롤링 시작
        overlays = ".se-module.se-module-text.se-title-text"                                 
        tit = driver.find_element_by_css_selector(overlays)          # title
        title = tit.text
        title = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9| ]", " ", title)
        

        # 글쓴이 크롤링 시작
        overlays = ".nick"                                 
        nick = driver.find_element_by_css_selector(overlays)         # nickname
        nickname = nick.text

        # 날짜 크롤링
        overlays = ".se_publishDate.pcol2"                                 
        date = driver.find_element_by_css_selector(overlays)         # datetime
        datetime = date.text

        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"                                 
        contents = driver.find_elements_by_css_selector(overlays)    # contents

        content_list = []
        for content in contents:
            content_list.append(content.text)
             
        content_str = ' '.join(content_list) # 하나 이상의 공백을 치환           # content_str
        content_str = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9|.|,| ]", " ", content_str)
        
        # 글 하나는 target_info라는 딕셔너리에 담기게 되고,
        target_info['title'] = title
        target_info['nickname'] = nickname
        target_info['datetime'] = datetime
        target_info['content'] = content_str

        # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
        dict[i] = target_info
        time.sleep(1)
        
        # 크롤링이 성공하면 글 제목을 출력하게 되고,
        print(i, title)

        # 글 하나 크롤링 후 크롬 창을 닫습니다.
        driver.close()       
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동합니다.
    except:
        driver.close()
        time.sleep(1)
        continue
    
    # 중간,중간에 파일로 저장하기
    if i == 100 or 300 or 500 :
        # 판다스로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_excel("blog_exercise_content10.xlsx", encoding='utf-8-sig')
        time.sleep(3)

print('수집한 글 갯수: ', len(dict))
print(dict)

end = time.time() - start
print("used time: %dm%ds" % divmod(end,60))